<a href="https://colab.research.google.com/github/boopkit/Classification/blob/master/TPK_Classification_Hackathon_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Honour Code
I Tamanique Kampman, confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

In [1]:
#Imports
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import datetime as dt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
train = pd.read_csv('training_1.csv')

In [0]:
test = pd.read_csv('test_1.csv')

In [4]:
df = pd.concat([train, test])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
df['Year'] = df['TransactionStartTime'].dt.year
df['Month'] = df['TransactionStartTime'].dt.month
df['Hour'] = df['TransactionStartTime'].dt.hour
df['Day'] = df['TransactionStartTime'].dt.day
df.drop('TransactionStartTime', axis=1, inplace=True)

In [0]:
numerical_to_cat = ['CountryCode', 'PricingStrategy']
df[numerical_to_cat] = df[numerical_to_cat].apply(lambda x: x.astype("str"))

In [0]:
from sklearn.preprocessing import LabelEncoder
for col in df.columns:
    if df[col].dtype=='object':
        l = LabelEncoder()
        l.fit(list(df[col].values))
        df[col] = l.transform(list(df[col].values))

In [8]:
df.head()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,Value,Year,Month,Hour,Day
0,3286,1000.0,68522,2,0,0,3786,0.0,2,0,1,5,4711,114984,1000,2018,11,2,15
1,4269,-20.0,45765,1,0,0,3786,0.0,2,2,23,3,3144,111539,20,2018,11,2,15
2,3589,500.0,88319,2,0,0,4093,0.0,2,0,0,5,1357,58688,500,2018,11,2,15
3,4451,20000.0,2629,2,0,0,7466,0.0,2,9,13,0,1318,71794,21800,2018,11,3,15
4,4269,-644.0,71473,1,0,0,7466,0.0,2,2,23,3,3144,60900,644,2018,11,3,15


In [0]:
train_df = df.iloc[:95662]
test_df = df.iloc[95662:].drop('FraudResult', axis=1)

In [0]:
X_train = train_df.drop('FraudResult', axis=1)
y_train = train_df['FraudResult']
X_test = test_df.values

In [11]:
from sklearn.ensemble import RandomForestClassifier

sm = SMOTE(random_state=42, ratio=1.0)
X, y = sm.fit_sample(X_train, y_train)
rf = RandomForestClassifier()
parameters_rf = {'min_samples_split' : [2, 4], 'n_estimators' : [50, 100]}
grid_rf = GridSearchCV(rf, parameters_rf, cv=5, verbose=1 , scoring ='roc_auc')
grid_rf.fit(X, y)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  8.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [0]:
preds = grid_rf.predict(X_test)
results = pd.DataFrame()
ids = test['TransactionId'].values
results['TransactionId'] = ids
results['FraudResult'] = preds
results.to_csv('sub_final.csv', index=False)